# Feature Enginering

## Feature Extraction

**Feature Extraction** o **Extracción de Características** es el proceso de derivar características útiles a partir de los datos originales que sean relevantes para el problema de **machine learning**. En PySpark, existen varias herramientas para extraer y generar nuevas características que aporten más valor al modelo, asegurando que la información más importante sea capturada de manera efectiva.

### 1. TF-IDF (Term Frequency - Inverse Document Frequency)

**TF-IDF** es una técnica en procesamiento de lenguaje natural y minería de textos que evalúa la importancia de una palabra en un documento en relación con un conjunto de documentos (corpus). Se utiliza ampliamente en tareas como la recuperación de información, el análisis de texto y la clasificación.

#### Componentes de TF-IDF

1. #### Term Frequency (TF) - Frecuencia de Término
   La **Frecuencia de Término** mide cuántas veces aparece una palabra en un documento específico. Es una medida de la frecuencia relativa de una palabra dentro de un documento particular.

   - **Fórmula**:
     ```
     TF = (Número de veces que aparece la palabra en el documento) / (Número total de palabras en el documento)
     ```

   - **Ejemplo**:
     Si la palabra "gato" aparece 3 veces en un documento de 100 palabras, entonces:
     ```
     TF = 3 / 100 = 0.03
     ```

2. #### Inverse Document Frequency (IDF) - Frecuencia Inversa de Documento
   La **Frecuencia Inversa de Documento** mide la importancia de una palabra dentro del corpus completo. Penaliza las palabras que aparecen en muchos documentos, dándoles menor peso, mientras que aumenta el peso de palabras que aparecen en pocos documentos.

   - **Fórmula**:
     ```
     IDF = log(Número total de documentos / Número de documentos que contienen la palabra)
     ```

   - **Ejemplo**:
     Si tenemos un corpus de 1000 documentos, y la palabra "gato" aparece en 10 de ellos, el IDF es:
     ```
     IDF = log(1000 / 10) = log(100) ≈ 2
     ```

3. #### TF-IDF
   La medida **TF-IDF** combina la frecuencia de término y la frecuencia inversa de documento para cada palabra en un documento específico. Multiplica **TF** e **IDF** para reflejar la relevancia de una palabra en un documento respecto al corpus.

   - **Fórmula**:
     ```
     TF-IDF = TF * IDF
     ```

   - **Interpretación**: 
     Un valor alto de TF-IDF indica que la palabra es relevante para ese documento en particular y poco común en otros documentos. Por el contrario, palabras comunes en el corpus tienen valores bajos de TF-IDF, ya que aportan poca relevancia específica.


#### Ejemplo de Cálculo de TF-IDF

Supongamos un corpus con 3 documentos:
- **Documento 1**: "El gato duerme en el sofá."
- **Documento 2**: "El perro duerme en la alfombra."
- **Documento 3**: "El gato y el perro juegan en el jardín."

Deseamos calcular el valor TF-IDF de la palabra "gato" en el Documento 1.

1. **Cálculo de TF** para "gato" en Documento 1:
   - "Gato" aparece una vez en un documento de 5 palabras.
   - ```
     TF = 1 / 5 = 0.2
     ```

2. **Cálculo de IDF** para "gato":
   - La palabra "gato" aparece en 2 de los 3 documentos.
   - ```
     IDF = log(3 / 2) ≈ 0.176
     ```

3. **Cálculo de TF-IDF**:
   - ```
     TF-IDF = 0.2 * 0.176 ≈ 0.0352
     ```

#### Aplicaciones de TF-IDF

- **Búsqueda de Información**: Ayuda a priorizar documentos que contienen términos relevantes para una consulta específica.
- **Clasificación de Textos**: Transforma el texto en vectores de características numéricas que pueden usarse en algoritmos de clasificación.
- **Análisis de Temas y Palabras Clave**: Permite identificar palabras clave representativas en documentos o temas específicos dentro de un corpus.

TF-IDF es ampliamente usado en tareas de análisis de texto y modelado en bibliotecas como **scikit-learn** y **PySpark**, donde el cálculo se realiza automáticamente usando herramientas como `TfidfVectorizer` o `HashingTF` y `IDF`.


In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer, IDF, Tokenizer
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, DoubleType, IntegerType
import pyspark.sql.functions as F
import numpy as np
sentenceData = spark.createDataFrame([(0, "Fútbol baloncesto tenis"),
                                      (1, "Fútbol tecnología IA"),
                                      (2, "Tenis baloncesto"),
                                      (3, "Tecnología innovación IA"),
                                      (4, "Fútbol deportes")
                                      ], ["document", "sentence"])
sentenceData.show(truncate=False)


+--------+------------------------+
|document|sentence                |
+--------+------------------------+
|0       |Fútbol baloncesto tenis |
|1       |Fútbol tecnología IA    |
|2       |Tenis baloncesto        |
|3       |Tecnología innovación IA|
|4       |Fútbol deportes         |
+--------+------------------------+



##### CountVectorizer

**CountVectorizer** es una técnica en procesamiento de lenguaje natural que convierte texto en una representación numérica basada en la frecuencia de palabras. Cada documento se transforma en un vector donde cada elemento representa la frecuencia de una palabra específica en ese documento.

##### Funcionamiento Básico

- `CountVectorizer` toma una colección de documentos y genera una **matriz de frecuencias de términos**.
- Cada fila representa un documento, y cada columna representa una palabra única en el corpus.
- Los valores en la matriz indican el número de veces que cada término aparece en cada documento.

##### Parámetros Clave

- **`vocabSize`**: Número máximo de términos únicos que se incluirán.
- **`minDF`**: Mínima frecuencia de documentos en los que debe aparecer un término para ser incluido.
- **`binary`**: Si es `True`, indica solo la presencia o ausencia de términos en lugar de la frecuencia.

##### Ejemplo

Para un corpus con dos documentos:
- Documento 1: "El perro juega en el parque"
- Documento 2: "El gato duerme en el sofá"

La matriz resultante podría verse así:

| Documento   | el | perro | juega | en | parque | gato | duerme | sofá |
|-------------|----|-------|-------|----|--------|------|--------|------|
| Documento 1 | 2  | 1     | 1     | 1  | 1      | 0    | 0      | 0    |
| Documento 2 | 2  | 0     | 0     | 1  | 0      | 1    | 1      | 1    |

`CountVectorizer` es útil para modelos de machine learning que requieren datos estructurados. En PySpark, se usa mediante la clase `pyspark.ml.feature.CountVectorizer`.


In [0]:
# Tokenizer: Divide la columna 'sentence' en palabras
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

# CountVectorizer: Convierte las palabras en una matriz de frecuencia de términos (TF)
vectorizer = CountVectorizer(inputCol="words", outputCol="rawFeatures")

# IDF: Calcula la frecuencia inversa de documento para los términos
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Pipeline: Agrupa todas las etapas en un flujo de trabajo
pipeline = Pipeline(stages=[tokenizer, vectorizer, idf])

# Entrena el modelo de Pipeline
model = pipeline.fit(sentenceData)

# Transforma los datos de entrada para ver los resultados de TF-IDF
result = model.transform(sentenceData)
print("Resultado de TF-IDF con el nuevo dataset:")
result.select("document", "sentence", "words", "rawFeatures", "features").show(truncate=False)


Resultado de TF-IDF con el nuevo dataset:
+--------+------------------------+----------------------------+-------------------------+----------------------------------------------------------------------+
|document|sentence                |words                       |rawFeatures              |features                                                              |
+--------+------------------------+----------------------------+-------------------------+----------------------------------------------------------------------+
|0       |Fútbol baloncesto tenis |[fútbol, baloncesto, tenis] |(7,[0,1,2],[1.0,1.0,1.0])|(7,[0,1,2],[0.4054651081081644,0.6931471805599453,0.6931471805599453])|
|1       |Fútbol tecnología IA    |[fútbol, tecnología, ia]    |(7,[0,3,4],[1.0,1.0,1.0])|(7,[0,3,4],[0.4054651081081644,0.6931471805599453,0.6931471805599453])|
|2       |Tenis baloncesto        |[tenis, baloncesto]         |(7,[1,2],[1.0,1.0])      |(7,[1,2],[0.6931471805599453,0.6931471805599453])         

In [0]:

# Calcular el conteo total de cada término en el corpus
total_counts = model.transform(sentenceData) \
    .select('rawFeatures').rdd \
    .map(lambda row: row['rawFeatures'].toArray()) \
    .reduce(lambda x, y: [x[i] + y[i] for i in range(len(y))])

# Obtener el vocabulario de CountVectorizer
vocabList = model.stages[1].vocabulary
d = {'vocabList': vocabList, 'counts': total_counts}

# Mostrar el vocabulario y sus frecuencias en el corpus
print("Vocabulario y conteo total de términos:")
spark.createDataFrame(np.array(list(d.values())).T.tolist(), list(d.keys())).show()


Vocabulario y conteo total de términos:
+----------+------+
| vocabList|counts|
+----------+------+
|    fútbol|   3.0|
|baloncesto|   2.0|
|     tenis|   2.0|
|tecnología|   2.0|
|        ia|   2.0|
|innovación|   1.0|
|  deportes|   1.0|
+----------+------+



In [0]:

# Mostrar el contenido de rawFeatures
counts = model.transform(sentenceData).select('rawFeatures').collect()
print("Conteos de términos en cada documento:")
print(counts)


Conteos de términos en cada documento:
[Row(rawFeatures=SparseVector(7, {0: 1.0, 1: 1.0, 2: 1.0})), Row(rawFeatures=SparseVector(7, {0: 1.0, 3: 1.0, 4: 1.0})), Row(rawFeatures=SparseVector(7, {1: 1.0, 2: 1.0})), Row(rawFeatures=SparseVector(7, {3: 1.0, 4: 1.0, 5: 1.0})), Row(rawFeatures=SparseVector(7, {0: 1.0, 6: 1.0}))]


In [0]:
# Función para convertir índices de términos en términos reales
def termsIdx2Term(vocabulary):
    def termsIdx2Term(termIndices):
        return [vocabulary[int(index)] for index in termIndices]
    return udf(termsIdx2Term, ArrayType(StringType()))

# Aplicar la función para convertir rawFeatures a términos legibles
vectorizerModel = model.stages[1]
vocabList = vectorizerModel.vocabulary
rawFeatures = model.transform(sentenceData).select('rawFeatures')

# Mostrar el DataFrame con índices, valores y términos
indices_udf = udf(lambda vector: vector.indices.tolist(), ArrayType(IntegerType()))
values_udf = udf(lambda vector: vector.toArray().tolist(), ArrayType(DoubleType()))

print("Raw features con índices, valores y términos correspondientes:")
rawFeatures.withColumn('indices', indices_udf(F.col('rawFeatures'))) \
           .withColumn('values', values_udf(F.col('rawFeatures'))) \
           .withColumn("Terms", termsIdx2Term(vocabList)("indices")) \
           .show(truncate=False)

Raw features con índices, valores y términos correspondientes:
+-------------------------+---------+-----------------------------------+----------------------------+
|rawFeatures              |indices  |values                             |Terms                       |
+-------------------------+---------+-----------------------------------+----------------------------+
|(7,[0,1,2],[1.0,1.0,1.0])|[0, 1, 2]|[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]|[fútbol, baloncesto, tenis] |
|(7,[0,3,4],[1.0,1.0,1.0])|[0, 3, 4]|[1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]|[fútbol, tecnología, ia]    |
|(7,[1,2],[1.0,1.0])      |[1, 2]   |[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]|[baloncesto, tenis]         |
|(7,[3,4,5],[1.0,1.0,1.0])|[3, 4, 5]|[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0]|[tecnología, ia, innovación]|
|(7,[0,6],[1.0,1.0])      |[0, 6]   |[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]|[fútbol, deportes]          |
+-------------------------+---------+-----------------------------------+----------------------------+




##### HashingTF

**HashingTF** es otra técnica para convertir texto en una representación numérica, pero en lugar de construir un vocabulario explícito, usa una función de **hashing** para mapear palabras a posiciones en un vector de tamaño fijo.

###### Funcionamiento Básico de HashingTF
- **Hashing**: En `HashingTF`, cada palabra se convierte en un índice en el vector mediante una función de hash, lo que permite definir el tamaño del vector (`numFeatures`) de antemano.
- **Colisiones de Hash**: Dado que usa hashing, diferentes palabras pueden mapearse a la misma posición, lo que genera colisiones. Esto puede reducir la precisión en algunos casos, pero mejora la eficiencia y la escalabilidad.

###### Diferencias entre CountVectorizer y HashingTF

| Aspecto               | CountVectorizer                          | HashingTF                                  |
|-----------------------|------------------------------------------|--------------------------------------------|
| **Vocabulario**       | Construye un vocabulario explícito       | No construye vocabulario, usa una función de hash |
| **Tamaño del Vector** | Depende del tamaño del vocabulario       | Fijo, determinado por `numFeatures`       |
| **Precisión**         | Alta (sin colisiones)                    | Puede sufrir colisiones                   |
| **Memoria**           | Puede ser intensivo en vocabularios grandes | Eficiente, adecuado para grandes conjuntos de datos |
| **Interpretabilidad** | Fácilmente interpretable                 | Menos interpretable debido a las colisiones |

##### ¿Cuándo usar cada uno?

- **Usa `CountVectorizer`** cuando el vocabulario no es extremadamente grande y necesitas precisión en la representación de los términos.
- **Usa `HashingTF`** en corpus grandes o cuando necesitas limitar el tamaño del vector para reducir el uso de memoria, aunque puede haber una ligera pérdida de precisión debido a colisiones.

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

# Crear un DataFrame de ejemplo
sentenceData = spark.createDataFrame([
    (0, "Python python Spark Spark"),
    (1, "Python SQL")
], ["document", "sentence"])

# Tokenizer: Divide la columna 'sentence' en palabras
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

# HashingTF: Convierte las palabras en una matriz de frecuencia de términos utilizando hashing
vectorizer = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=5)

# IDF: Calcula la frecuencia inversa de documentos para los términos
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Pipeline: Agrupa todas las etapas en un flujo de trabajo
pipeline = Pipeline(stages=[tokenizer, vectorizer, idf])

# Entrena el modelo de Pipeline
model = pipeline.fit(sentenceData)

# Transforma los datos de entrada para ver los resultados de TF-IDF
result = model.transform(sentenceData)

# Muestra el resultado final
result.show(truncate=False)


+--------+-------------------------+------------------------------+-------------------+----------------------------------+
|document|sentence                 |words                         |rawFeatures        |features                          |
+--------+-------------------------+------------------------------+-------------------+----------------------------------+
|0       |Python python Spark Spark|[python, python, spark, spark]|(5,[1,4],[2.0,2.0])|(5,[1,4],[0.8109302162163288,0.0])|
|1       |Python SQL               |[python, sql]                 |(5,[2,4],[1.0,1.0])|(5,[2,4],[0.4054651081081644,0.0])|
+--------+-------------------------+------------------------------+-------------------+----------------------------------+



### 2. Word2Vec

**Word2Vec** es un modelo de aprendizaje profundo que convierte palabras en vectores numéricos, capturando relaciones semánticas entre ellas. En Spark, `Word2Vec` toma un conjunto de documentos tokenizados y asigna a cada palabra un vector en un espacio de características, donde palabras con significados similares tienen representaciones vectoriales cercanas.

##### Funcionamiento Básico
- `Word2Vec` aprende una representación densa para cada palabra en función de su contexto en una ventana de palabras (tamaño definido por el parámetro `windowSize`).
- Los vectores de palabras pueden usarse en tareas de NLP como análisis de similitud o clustering.

##### Parámetros Clave
- **`vectorSize`**: Define el tamaño del vector resultante para cada palabra.
- **`windowSize`**: Especifica el tamaño de la ventana de contexto utilizada para el entrenamiento.
- **`minCount`**: Número mínimo de ocurrencias de una palabra para incluirla en el modelo.

En PySpark, el modelo `Word2Vec` se encuentra en `pyspark.ml.feature.Word2Vec` y se entrena usando un pipeline para obtener representaciones vectoriales de palabras en textos grandes.


#### Word Embedding Models in PySpark

In [0]:
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import Tokenizer
from pyspark.ml import Pipeline

# Ejemplo de datos: sentenceData
sentenceData = spark.createDataFrame([
    (0, "El fútbol es un deporte muy popular"),
    (1, "La tecnología avanza rápidamente"),
    (2, "Los deportes y la tecnología son temas de interés")
], ["document", "sentence"])

# Tokenizer: Convierte la columna 'sentence' en una lista de palabras 'words'
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

# Word2Vec: Convierte las palabras en vectores de características
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="words", outputCol="feature")

# Pipeline: Agrupa las etapas de Tokenizer y Word2Vec en un flujo de trabajo
pipeline = Pipeline(stages=[tokenizer, word2Vec])

# Entrena el modelo de Word2Vec
model = pipeline.fit(sentenceData)

# Transforma los datos de entrada para obtener los vectores de características
result = model.transform(sentenceData)

# Muestra el resultado final
result.select("document", "sentence", "words", "feature").show(truncate=False)


+--------+-------------------------------------------------+-----------------------------------------------------------+-----------------------------------------------------------------+
|document|sentence                                         |words                                                      |feature                                                          |
+--------+-------------------------------------------------+-----------------------------------------------------------+-----------------------------------------------------------------+
|0       |El fútbol es un deporte muy popular              |[el, fútbol, es, un, deporte, muy, popular]                |[-0.03954152097659452,-0.008140003042561668,-0.01849026765142168]|
|1       |La tecnología avanza rápidamente                 |[la, tecnología, avanza, rápidamente]                      |[-0.0469239829108119,0.03239990258589387,0.026472446508705616]   |
|2       |Los deportes y la tecnología son temas de interés|[los,

In [0]:
# Obtener los vectores de cada palabra en el modelo Word2Vec
w2v = model.stages[1]  # Accede a la etapa de Word2Vec en el pipeline
print("Vectores de palabras individuales:")
w2v.getVectors().show(truncate=False)

Vectores de palabras individuales:
+-----------+------------------------------------------------------------------+
|word       |vector                                                            |
+-----------+------------------------------------------------------------------+
|muy        |[-0.15639916062355042,-0.14967060089111328,-0.12522591650485992]  |
|tecnología |[-0.03242826089262962,0.025772446766495705,0.03958774730563164]   |
|y          |[-0.02910229191184044,0.14060592651367188,0.010431595146656036]   |
|interés    |[0.0013236792292445898,0.010991252027451992,-0.11285977810621262] |
|temas      |[0.09195507317781448,0.14056122303009033,-0.008430935442447662]   |
|fútbol     |[-0.08341887593269348,0.0737525001168251,0.12147324532270432]     |
|el         |[-0.13390909135341644,0.03708652779459953,0.07401996105909348]    |
|la         |[-0.09924070537090302,0.015769578516483307,0.03510101139545441]   |
|deportes   |[0.12654274702072144,0.06648338586091995,0.13824526965618134]

### 3. FeatureHasher 

**FeatureHasher** es una técnica en Spark para convertir varias columnas de diferentes tipos (numéricas, categóricas y booleanas) en un único vector de características, utilizando un método de **hashing**. Esto es útil para transformar datos de alto volumen y reducir el espacio de características, especialmente en conjuntos de datos que incluyen columnas categóricas de gran tamaño.

#### Funcionamiento Básico

- **Hashing**: Cada valor en las columnas de entrada se convierte en una posición en el vector de salida usando una función de hash. Esto permite representar grandes volúmenes de datos categóricos en un espacio vectorial de tamaño fijo.
- **Vector de Características**: `FeatureHasher` produce un vector disperso de tamaño fijo en la columna de salida especificada. Las entradas numéricas se copian directamente al vector, mientras que las entradas categóricas y booleanas se convierten en índices de características mediante hashing.

#### Parámetros Clave

- **`inputCols`**: Lista de las columnas de entrada que se van a transformar.
- **`outputCol`**: Nombre de la columna de salida que contendrá el vector de características.
- **`numFeatures`** (opcional): Número de características en el vector de salida. Si no se especifica, Spark utiliza un valor predeterminado.


In [0]:
from pyspark.ml.feature import FeatureHasher

# Nuevo dataset de ejemplo sobre productos
dataset = spark.createDataFrame([
    (12.5, True, "101", "electronics"),
    (8.3, False, "102", "clothing"),
    (15.7, True, "103", "furniture"),
    (3.2, False, "104", "electronics"),
    (7.5, True, "105", "groceries"),
    (6.0, False, "106", "clothing")
], ["price", "in_stock", "product_code", "category"])

# Definir el FeatureHasher
hasher = FeatureHasher(inputCols=["price", "in_stock", "product_code", "category"], outputCol="features")

# Transformar el DataFrame
featurized = hasher.transform(dataset)

# Mostrar el resultado
featurized.show(truncate=False)


+-----+--------+------------+-----------+--------------------------------------------------------+
|price|in_stock|product_code|category   |features                                                |
+-----+--------+------------+-----------+--------------------------------------------------------+
|12.5 |true    |101         |electronics|(262144,[16758,162583,201386,211061],[1.0,1.0,12.5,1.0])|
|8.3  |false   |102         |clothing   |(262144,[61126,123535,183867,201386],[1.0,1.0,1.0,8.3]) |
|15.7 |true    |103         |furniture  |(262144,[59816,148779,162583,201386],[1.0,1.0,1.0,15.7])|
|3.2  |false   |104         |electronics|(262144,[183867,199043,201386,211061],[1.0,1.0,3.2,1.0])|
|7.5  |true    |105         |groceries  |(262144,[23793,162583,201386,259376],[1.0,1.0,7.5,1.0]) |
|6.0  |false   |106         |clothing   |(262144,[21292,61126,183867,201386],[1.0,1.0,1.0,6.0])  |
+-----+--------+------------+-----------+--------------------------------------------------------+



### 3. RFormula

**RFormula** es una herramienta en PySpark para transformar datos en un formato adecuado para machine learning, inspirada en la sintaxis de fórmulas de **R**. `RFormula` permite especificar relaciones entre características (features) y etiquetas (label) de una manera concisa, y es útil para crear automáticamente vectores de características y convertir variables categóricas en variables numéricas.

#### Funcionamiento Básico

- **Fórmula**: La fórmula de `RFormula` sigue el estilo de R y tiene la forma `label ~ feature1 + feature2 + ...`. Aquí, `label` es la variable objetivo (dependiente) y `feature1`, `feature2`, etc., son las variables de entrada (independientes).
- **Codificación de Variables Categóricas**: `RFormula` convierte automáticamente las columnas categóricas en representaciones numéricas usando codificación de una sola categoría (one-hot encoding), si es necesario.
- **Vector de Características**: Las características se agrupan en un único vector llamado `features`, y la columna de salida para la etiqueta se llama `label`.


In [0]:
from pyspark.ml.feature import RFormula
# Crear un DataFrame de ejemplo
dataset = spark.createDataFrame([
    (1, "M", 25, 3000.0),
    (2, "F", 30, 4000.0),
    (3, "M", 35, 5000.0),
    (4, "F", 40, 6000.0)
], ["id", "gender", "age", "income"])

# Definir la fórmula: "income" es la etiqueta y "gender" y "age" son características
formula = RFormula(formula="income ~ gender + age", featuresCol="features", labelCol="label")

# Transformar el DataFrame
output = formula.fit(dataset).transform(dataset)

# Mostrar el resultado
output.select("id", "gender", "age", "income", "features", "label").show(truncate=False)

+---+------+---+------+----------+------+
|id |gender|age|income|features  |label |
+---+------+---+------+----------+------+
|1  |M     |25 |3000.0|[0.0,25.0]|3000.0|
|2  |F     |30 |4000.0|[1.0,30.0]|4000.0|
|3  |M     |35 |5000.0|[0.0,35.0]|5000.0|
|4  |F     |40 |6000.0|[1.0,40.0]|6000.0|
+---+------+---+------+----------+------+



## Feature Transform

**Feature Transformers** son técnicas en **PySpark** utilizadas para **transformar las características** (features) de los datos, generalmente como parte del preprocesamiento antes de aplicar algoritmos de machine learning. Estos transformadores permiten ajustar, modificar y mejorar los datos para que los algoritmos de aprendizaje automático puedan procesarlos de manera más eficiente y precisa.



### 1. Tokenizer

**Tokenizer** es una técnica en procesamiento de lenguaje natural (NLP) que consiste en dividir un texto en unidades más pequeñas llamadas *tokens*. Estos *tokens* suelen ser palabras individuales, pero también pueden ser frases o caracteres. El proceso de tokenización convierte una secuencia de texto en una secuencia de *tokens*, que luego puede analizarse matemáticamente en tareas de NLP.

#### Tipos de Tokenizadores en PySpark

1. **Tokenizer**:
   - **Función**: Divide el texto en palabras, usando espacios en blanco como delimitador predeterminado. Cada palabra se convierte en un token.
   - **Aplicación**: Es ideal para textos donde las palabras están separadas por espacios y no se requiere procesar caracteres especiales o puntuación.
   - **Ejemplo**:
     - Entrada: `"Hi I heard about Spark"`
     - Salida: `["Hi", "I", "heard", "about", "Spark"]`

2. **RegexTokenizer**:
   - **Función**: Permite un control más avanzado en la tokenización, usando expresiones regulares para definir cómo separar los tokens.
   - **Aplicación**: Se utiliza cuando se necesita una tokenización más compleja, por ejemplo, para dividir por caracteres especiales, eliminar puntuación, o controlar el tratamiento de espacios.
   - **Patrón Común**: `pattern="\\W"` separa el texto en palabras ignorando todos los caracteres que no sean alfanuméricos, eliminando signos de puntuación.
   - **Ejemplo**:
     - Entrada: `"Logistic,regression,models,are,neat"`
     - Salida con `pattern="\\W"`: `["Logistic", "regression", "models", "are", "neat"]`


In [0]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

# Nuevo DataFrame de ejemplo
sentenceDataFrame = spark.createDataFrame([
    (0, "La ciencia de datos es increíble!"),
    (1, "Machine-learning, IA y datos"),
    (2, "Análisis de Big-data es esencial.")
], ["id", "sentence"])

# Definir Tokenizer (basado en espacios)
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

# Definir RegexTokenizer (basado en expresiones regulares para ignorar caracteres no alfanuméricos)
regexTokenizer = RegexTokenizer(inputCol="sentence", outputCol="words", pattern="\\W")

# UDF para contar el número de tokens
countTokens = udf(lambda words: len(words), IntegerType())

# Tokenizar y contar tokens con Tokenizer
tokenized = tokenizer.transform(sentenceDataFrame)
tokenized.select("sentence", "words")\
         .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

# Tokenizar y contar tokens con RegexTokenizer
regexTokenized = regexTokenizer.transform(sentenceDataFrame)
regexTokenized.select("sentence", "words")\
              .withColumn("tokens", countTokens(col("words"))).show(truncate=False)


+---------------------------------+----------------------------------------+------+
|sentence                         |words                                   |tokens|
+---------------------------------+----------------------------------------+------+
|La ciencia de datos es increíble!|[la, ciencia, de, datos, es, increíble!]|6     |
|Machine-learning, IA y datos     |[machine-learning,, ia, y, datos]       |4     |
|Análisis de Big-data es esencial.|[análisis, de, big-data, es, esencial.] |5     |
+---------------------------------+----------------------------------------+------+

+---------------------------------+----------------------------------------+------+
|sentence                         |words                                   |tokens|
+---------------------------------+----------------------------------------+------+
|La ciencia de datos es increíble!|[la, ciencia, de, datos, es, incre, ble]|7     |
|Machine-learning, IA y datos     |[machine, learning, ia, y, datos]       

### 2. StopWordsRemover

**StopWordsRemover** es una herramienta en PySpark que elimina *stop words* de una columna de texto tokenizado. Las *stop words* son palabras comunes en un idioma (como "y", "el", "de" en español o "and", "the", "of" en inglés) que tienen poco valor informativo y suelen eliminarse en el procesamiento de texto para reducir el ruido y mejorar la calidad del análisis.

#### ¿Cómo Funciona?

1. **Entrada de Texto Tokenizado**:
   - `StopWordsRemover` requiere una columna de texto ya tokenizado, es decir, una columna donde el texto esté dividido en palabras individuales o *tokens*. Esto significa que normalmente se aplica después de una etapa de tokenización, como `Tokenizer` o `RegexTokenizer`.

2. **Eliminación de Stop Words**:
   - El transformador elimina los tokens que coinciden con palabras de una lista de *stop words* predefinida en PySpark para un idioma específico.
   - PySpark incluye listas de *stop words* en varios idiomas, y también permite definir una lista personalizada.

3. **Salida**:
   - El `StopWordsRemover` genera una nueva columna con las *stop words* eliminadas, dejando solo las palabras informativas.


In [0]:
from pyspark.ml.feature import StopWordsRemover

# Crear un DataFrame de ejemplo con frases en español
sentenceData = spark.createDataFrame([
    (0, ["me", "gusta", "la", "naturaleza", "y", "los", "animales"]),
    (1, ["es", "importante", "cuidar", "el", "medio", "ambiente"]),
    (2, ["la", "tecnología", "avanza", "muy", "rápido"])
], ["id", "raw"])

# Configurar StopWordsRemover con stop words en español
remover = StopWordsRemover(inputCol="raw", outputCol="removed", stopWords=["me", "la", "y", "los", "es", "el", "muy"])

# Aplicar StopWordsRemover y mostrar el resultado
cleanedData = remover.transform(sentenceData)
cleanedData.select("raw", "removed").show(truncate=False)


+---------------------------------------------+-------------------------------------+
|raw                                          |removed                              |
+---------------------------------------------+-------------------------------------+
|[me, gusta, la, naturaleza, y, los, animales]|[gusta, naturaleza, animales]        |
|[es, importante, cuidar, el, medio, ambiente]|[importante, cuidar, medio, ambiente]|
|[la, tecnología, avanza, muy, rápido]        |[tecnología, avanza, rápido]         |
+---------------------------------------------+-------------------------------------+



### 3. NGram


**NGram** es una técnica en procesamiento de lenguaje natural (NLP) que agrupa palabras consecutivas en secuencias de longitud `n`, conocidas como *n-gramas*. Estas secuencias de palabras permiten capturar patrones y dependencias en el texto, lo cual es especialmente útil en modelos de lenguaje y en tareas de clasificación de texto. 

En PySpark, el transformador `NGram` toma una columna de texto tokenizado y genera una nueva columna con las secuencias de *n-gramas*.

#### ¿Qué es un N-Grama?

Un *n-grama* es una secuencia de `n` palabras consecutivas en una frase o documento. 
- Cuando `n=1`, se llama **unigrama** y cada token es una sola palabra.
- Cuando `n=2`, se llama **bigrama** y cada token es una secuencia de dos palabras consecutivas.
- Cuando `n=3`, se llama **trigrama** y cada token es una secuencia de tres palabras consecutivas.

Por ejemplo, en la frase `"El análisis de datos es importante"`:
- Los bigramas serían: `["El análisis", "análisis de", "de datos", "datos es", "es importante"]`
- Los trigramas serían: `["El análisis de", "análisis de datos", "de datos es", "datos es importante"]`

#### Importancia de los N-Gramas

- **Contexto de Palabras**: Los *n-gramas* ayudan a capturar el contexto en el que las palabras aparecen, lo cual es crucial para modelos de lenguaje, análisis de sentimiento y clasificación de texto.
- **Modelado de Dependencias**: Los *n-gramas* permiten modelar dependencias locales en el texto, ayudando a identificar patrones o expresiones frecuentes.
- **Preparación de Datos para Modelos**: En el aprendizaje automático, los *n-gramas* permiten que los modelos de NLP capturen relaciones entre palabras en función de su proximidad, mejorando la precisión del análisis de texto.


In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, NGram
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("NGramExample").getOrCreate()

# DataFrame de ejemplo con frases en español
sentenceData = spark.createDataFrame([
    (0.0, "Me encanta Spark"),
    (0.0, "Me encanta Python"),
    (1.0, "El aprendizaje es fascinante")
], ["label", "sentence"])

# Tokenizer para dividir las frases en palabras
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

# NGram para generar bigramas (n=2)
ngram = NGram(n=2, inputCol="words", outputCol="ngrams")

# Definir el pipeline con las etapas de tokenización y generación de bigramas
pipeline = Pipeline(stages=[tokenizer, ngram])

# Entrenar el modelo
model = pipeline.fit(sentenceData)

# Transformar los datos y mostrar el resultado
model.transform(sentenceData).select("sentence", "words", "ngrams").show(truncate=False)

+----------------------------+---------------------------------+-----------------------------------------------+
|sentence                    |words                            |ngrams                                         |
+----------------------------+---------------------------------+-----------------------------------------------+
|Me encanta Spark            |[me, encanta, spark]             |[me encanta, encanta spark]                    |
|Me encanta Python           |[me, encanta, python]            |[me encanta, encanta python]                   |
|El aprendizaje es fascinante|[el, aprendizaje, es, fascinante]|[el aprendizaje, aprendizaje es, es fascinante]|
+----------------------------+---------------------------------+-----------------------------------------------+



### 4. Binarizer

**Binarizer** es una herramienta en PySpark que convierte valores numéricos en valores binarios (0 o 1) en función de un umbral definido. Se utiliza en machine learning para transformar características continuas en características binarias, permitiendo que el modelo trate los datos en categorías simples de presencia o ausencia. 

#### Funcionamiento

- **Umbral (`threshold`)**: `Binarizer` asigna el valor `1.0` a los datos que son iguales o superiores al umbral, y el valor `0.0` a los datos que son inferiores al umbral.
- **Entrada y Salida**: Toma como entrada una columna numérica y genera una nueva columna con los valores binarizados.

Por ejemplo, si aplicamos un umbral de `3.0` a la lista `[2.5, 3.5, 4.0, 2.0]`, el resultado será `[0, 1, 1, 0]`, ya que solo los valores mayores o iguales a `3.0` se convierten en `1.0`.


In [0]:
from pyspark.ml.feature import Binarizer

# Crear DataFrame de ejemplo con valores continuos en la columna "feature"
continuousDataFrame = spark.createDataFrame([
    (0, 0.1),
    (1, 0.8),
    (2, 0.2),
    (3, 0.5)
], ["id", "feature"])

# Configurar Binarizer para convertir "feature" a valores binarios en "binarized_feature" usando un umbral de 0.5
binarizer = Binarizer(threshold=0.5, inputCol="feature", outputCol="binarized_feature")

# Transformar el DataFrame
binarizedDataFrame = binarizer.transform(continuousDataFrame)

# Mostrar el umbral utilizado y el DataFrame resultante
print("Binarizer output with Threshold = %f" % binarizer.getThreshold())
binarizedDataFrame.show()


Binarizer output with Threshold = 0.500000
+---+-------+-----------------+
| id|feature|binarized_feature|
+---+-------+-----------------+
|  0|    0.1|              0.0|
|  1|    0.8|              1.0|
|  2|    0.2|              0.0|
|  3|    0.5|              0.0|
+---+-------+-----------------+



### 5. StringIndexer

**StringIndexer** es una herramienta en PySpark que convierte columnas categóricas con valores de texto en valores numéricos, asignando un índice único a cada categoría distinta. Esta transformación es esencial para el preprocesamiento de datos en machine learning, ya que muchos algoritmos requieren variables numéricas en lugar de texto.

#### Funcionamiento

1. **Asignación de Índices**: `StringIndexer` toma cada valor de texto en la columna de entrada y le asigna un índice numérico en función de la frecuencia de aparición:
   - La categoría con mayor frecuencia recibe el índice `0`.
   - La siguiente categoría más frecuente recibe el índice `1`, y así sucesivamente.

2. **Columna de Salida**: La columna de salida contiene los índices correspondientes a cada valor de texto en la columna original, permitiendo que los algoritmos de machine learning trabajen con datos categóricos en formato numérico.


In [0]:
from pyspark.ml.feature import StringIndexer

# Crear DataFrame de ejemplo con columna categórica "category"
df = spark.createDataFrame([
    (0, "a"), 
    (1, "b"), 
    (2, "c"), 
    (3, "a"), 
    (4, "a"), 
    (5, "c")
], ["id", "category"])

# Configurar StringIndexer para transformar "category" a índices numéricos en "categoryIndex"
indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")

# Ajustar y transformar el DataFrame
indexed = indexer.fit(df).transform(df)

# Mostrar el DataFrame resultante
indexed.show()


+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+



### 6. LabelConverter

**LabelConverter**, implementado en PySpark como `IndexToString`, es una herramienta utilizada para revertir la transformación de índices numéricos a etiquetas de texto originales en datos categóricos. Esto es útil en machine learning, especialmente cuando se han convertido etiquetas de texto a índices numéricos para entrenar el modelo, pero queremos interpretar y presentar los resultados finales en términos de sus categorías originales.

#### Funcionamiento

1. **Conversión Inicial con `StringIndexer`**:
   - En muchos casos, antes de entrenar un modelo de machine learning, se convierte la columna de etiquetas de texto en índices numéricos con `StringIndexer`. Esto facilita que el modelo trabaje con datos categóricos.
   
2. **Reversión de Índices con `IndexToString`**:
   - Una vez finalizado el entrenamiento y obtenidos los resultados (por ejemplo, predicciones), `IndexToString` permite revertir los índices numéricos de vuelta a las etiquetas de texto originales. Esto es esencial para hacer que los resultados sean interpretables para el usuario.


In [0]:
from pyspark.ml.feature import IndexToString, StringIndexer

# Crear un DataFrame de ejemplo con una columna de etiquetas de texto ("Yes" y "No")
df = spark.createDataFrame([
    (0, "Yes"), 
    (1, "Yes"), 
    (2, "Yes"), 
    (3, "No"), 
    (4, "No"), 
    (5, "No")
], ["id", "label"])

# Convertir la columna de etiquetas de texto a índices numéricos
indexer = StringIndexer(inputCol="label", outputCol="labelIndex")
model = indexer.fit(df)
indexed = model.transform(df)
print("Transformed string column '%s' to indexed column '%s'" % \
    (indexer.getInputCol(), indexer.getOutputCol()))
indexed.show()

# Configurar IndexToString para revertir la columna de índices a los valores de texto originales
converter = IndexToString(inputCol="labelIndex", outputCol="originalLabel")
converted = converter.transform(indexed)

# Mostrar el resultado de la conversión de vuelta a las etiquetas de texto originales
print("Transformed indexed column '%s' back to original string column '%s' using "\
    "labels in metadata" % (converter.getInputCol(), converter.getOutputCol()))
converted.select("id", "labelIndex", "originalLabel").show()


Transformed string column 'label' to indexed column 'labelIndex'
+---+-----+----------+
| id|label|labelIndex|
+---+-----+----------+
|  0|  Yes|       1.0|
|  1|  Yes|       1.0|
|  2|  Yes|       1.0|
|  3|   No|       0.0|
|  4|   No|       0.0|
|  5|   No|       0.0|
+---+-----+----------+

Transformed indexed column 'labelIndex' back to original string column 'originalLabel' using labels in metadata
+---+----------+-------------+
| id|labelIndex|originalLabel|
+---+----------+-------------+
|  0|       1.0|          Yes|
|  1|       1.0|          Yes|
|  2|       1.0|          Yes|
|  3|       0.0|           No|
|  4|       0.0|           No|
|  5|       0.0|           No|
+---+----------+-------------+



In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString, StringIndexer

# Crear un DataFrame de ejemplo con una columna de etiquetas ("Yes" y "No")
df = spark.createDataFrame([
    (0, "Yes"), 
    (1, "Yes"), 
    (2, "Yes"), 
    (3, "No"), 
    (4, "No"), 
    (5, "No")
], ["id", "label"])

# Configurar StringIndexer para convertir la columna de texto "label" en índices numéricos
indexer = StringIndexer(inputCol="label", outputCol="labelIndex")

# Configurar IndexToString para convertir los índices de vuelta a las etiquetas originales
converter = IndexToString(inputCol="labelIndex", outputCol="originalLabel")

# Crear un pipeline con las etapas de indexación y conversión
pipeline = Pipeline(stages=[indexer, converter])

# Ajustar el modelo y aplicar la transformación al DataFrame
model = pipeline.fit(df)
result = model.transform(df)

# Mostrar el resultado
result.show()


+---+-----+----------+-------------+
| id|label|labelIndex|originalLabel|
+---+-----+----------+-------------+
|  0|  Yes|       1.0|          Yes|
|  1|  Yes|       1.0|          Yes|
|  2|  Yes|       1.0|          Yes|
|  3|   No|       0.0|           No|
|  4|   No|       0.0|           No|
|  5|   No|       0.0|           No|
+---+-----+----------+-------------+



### 7. VectorIndexer

**VectorIndexer** es una herramienta en PySpark que ayuda a identificar y transformar automáticamente columnas categóricas en un vector de características. Este transformador es especialmente útil cuando se trabaja con datos en forma de vectores que contienen tanto características numéricas continuas como categóricas. Al indexar automáticamente las columnas categóricas en un vector, `VectorIndexer` facilita el preprocesamiento de datos para algoritmos de machine learning que requieren categorías representadas numéricamente.

#### ¿Cómo Funciona?

1. **Identificación Automática de Columnas Categóricas**:
   - `VectorIndexer` analiza las características en el vector y considera una característica como categórica si tiene un número de valores distintos menor o igual a un umbral (`maxCategories`). Este parámetro se puede ajustar según el conjunto de datos.

2. **Indexación de Columnas Categóricas**:
   - Para cada columna categórica identificada, `VectorIndexer` asigna un índice numérico a cada categoría. Esto convierte valores categóricos en valores numéricos que los modelos de machine learning pueden procesar.

3. **Compatibilidad con Algoritmos de Machine Learning**:
   - La indexación de categorías es útil para algoritmos que no pueden trabajar directamente con datos categóricos, como los árboles de decisión y otros modelos basados en reglas.


In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer, RFormula
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("VectorIndexerPeruExample").getOrCreate()

# Crear un DataFrame de ejemplo con características de distintos tipos, usando departamentos de Perú
df = spark.createDataFrame([
    (0, 1.5, True, "3", "turismo", "Lima"),
    (1, 2.5, False, "5", "industria", "Arequipa"),
    (0, 4.0, True, "4", "agricultura", "Cusco"),
    (1, 3.2, False, "2", "minería", "Puno"),
    (0, 5.0, True, "6", "pesca", "Piura")
], ['label', "pib", "costa", "ranking", "sector", "departamento"])

# Configurar RFormula para generar automáticamente el vector de características
formula = RFormula(
    formula="label ~ pib + costa + ranking + sector + departamento",
    featuresCol="features",
    labelCol="label"
)

# Configurar VectorIndexer para identificar y convertir características categóricas
# maxCategories=2 considera como categóricas solo las columnas con <= 2 valores distintos
featureIndexer = VectorIndexer(
    inputCol="features",
    outputCol="indexedFeatures",
    maxCategories=2
)

# Crear un pipeline con RFormula y VectorIndexer
pipeline = Pipeline(stages=[formula, featureIndexer])

# Ajustar el pipeline y transformar el DataFrame
model = pipeline.fit(df)
result = model.transform(df)

# Mostrar el DataFrame transformado
result.select("label", "features", "indexedFeatures").show(truncate=False)


+-----+---------------------------------------+---------------------------------------+
|label|features                               |indexedFeatures                        |
+-----+---------------------------------------+---------------------------------------+
|0    |(14,[0,1,3,12],[1.5,1.0,1.0,1.0])      |(14,[0,1,3,12],[1.5,1.0,1.0,1.0])      |
|1    |(14,[0,5,7,10],[2.5,1.0,1.0,1.0])      |(14,[0,5,7,10],[2.5,1.0,1.0,1.0])      |
|0    |(14,[0,1,4,6,11],[4.0,1.0,1.0,1.0,1.0])|(14,[0,1,4,6,11],[4.0,1.0,1.0,1.0,1.0])|
|1    |(14,[0,2,8],[3.2,1.0,1.0])             |(14,[0,2,8],[3.2,1.0,1.0])             |
|0    |(14,[0,1,9,13],[5.0,1.0,1.0,1.0])      |(14,[0,1,9,13],[5.0,1.0,1.0,1.0])      |
+-----+---------------------------------------+---------------------------------------+



### 8. VectorAssembler

**VectorAssembler** es una herramienta en PySpark que combina múltiples columnas en un solo **vector de características**. Esto es útil en machine learning cuando se necesita consolidar varias características en un único vector para alimentar modelos de aprendizaje automático. **VectorAssembler** puede manejar tanto columnas numéricas como vectoriales.

#### Funcionamiento

1. **Agrupación de Columnas**:
   - **VectorAssembler** toma varias columnas (numéricas o vectoriales) de un `DataFrame` y las concatena en una única columna de vector de salida, permitiendo que todas las características se representen en un solo vector.

2. **Preparación para Modelos**:
   - La mayoría de los modelos de PySpark requieren una columna de características unificada. **VectorAssembler** facilita esto y asegura la compatibilidad de los datos.



In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Crear un DataFrame de ejemplo con características relacionadas con datos de telefonía
dataset = spark.createDataFrame([
    (0, 300, 1.0, Vectors.dense([5.0, 2.0, 10.0]), 1),  # 300 segundos de llamada, usuario en plan pospago, actividad de usuario
    (1, 50, 0.0, Vectors.dense([2.0, 1.0, 0.0]), 0),    # 50 segundos de llamada, usuario en plan prepago, actividad de usuario
    (2, 600, 1.0, Vectors.dense([15.0, 5.0, 25.0]), 1)  # 600 segundos de llamada, usuario en plan pospago, actividad de usuario
], ["id", "duracionLlamada", "tipoPlan", "actividadUsuario", "realizoCompra"])

# Configurar VectorAssembler para combinar las columnas en un vector de características
assembler = VectorAssembler(
    inputCols=["duracionLlamada", "tipoPlan", "actividadUsuario"],
    outputCol="caracteristicas"
)

# Transformar el DataFrame
output = assembler.transform(dataset)
print("Columnas ensambladas 'duracionLlamada', 'tipoPlan', 'actividadUsuario' en la columna de vector 'caracteristicas'")
output.select("caracteristicas", "realizoCompra").show(truncate=False)


Columnas ensambladas 'duracionLlamada', 'tipoPlan', 'actividadUsuario' en la columna de vector 'caracteristicas'
+-------------------------+-------------+
|caracteristicas          |realizoCompra|
+-------------------------+-------------+
|[300.0,1.0,5.0,2.0,10.0] |1            |
|[50.0,0.0,2.0,1.0,0.0]   |0            |
|[600.0,1.0,15.0,5.0,25.0]|1            |
+-------------------------+-------------+



### 9. OneHotEncoder

**OneHotEncoder** es una técnica de codificación de variables categóricas que convierte cada categoría en un **vector binario**. Este vector tiene un valor de `1` en la posición de la categoría correspondiente y `0` en todas las demás posiciones. **OneHotEncoder** es ampliamente utilizado en machine learning, ya que muchos algoritmos requieren datos en formato numérico o binario.

#### ¿Cómo Funciona?

1. **Conversión de Categorías a Índices**:
   - Para aplicar **OneHotEncoder**, primero es necesario convertir las categorías en índices numéricos utilizando `StringIndexer`. Cada categoría en una columna es asignada a un índice único.
   
2. **Codificación One-Hot**:
   - Una vez convertidas las categorías a índices, **OneHotEncoder** genera un vector binario donde cada posición corresponde a una categoría. Si la categoría de una observación está presente en una columna, se le asigna `1` en su posición, y `0` en las demás.
   
3. **Salida en Formato Vectorial**:
   - **OneHotEncoder** produce una columna de salida con un vector para cada observación, representando la presencia de una categoría específica de manera numérica y binaria. Esto permite que los datos categóricos sean compatibles con modelos de machine learning que no pueden trabajar directamente con datos de texto.

#### Ejemplo Rápido de OneHotEncoder

Supongamos que tenemos una columna de datos con tipos de frutas:

- **Manzana**
- **Naranja**
- **Banana**

Después de aplicar `StringIndexer`, los valores pueden quedar así:
- **Manzana** → 0
- **Naranja** → 1
- **Banana** → 2

Con **OneHotEncoder**, estos índices se transforman en vectores:
- **Manzana** → `[1.0, 0.0, 0.0]`
- **Naranja** → `[0.0, 1.0, 0.0]`
- **Banana** → `[0.0, 0.0, 1.0]`

### Visualización del Proceso

1. **Datos Originales**:
   | Fruta    |
   |----------|
   | Manzana  |
   | Naranja  |
   | Banana   |

2. **Transformación con StringIndexer**:
   | Fruta    | Índice |
   |----------|--------|
   | Manzana  | 0      |
   | Naranja  | 1      |
   | Banana   | 2      |

3. **Codificación One-Hot**:
   | Fruta    | Vector One-Hot        |
   |----------|-----------------------|
   | Manzana  | `[1.0, 0.0, 0.0]`     |
   | Naranja  | `[0.0, 1.0, 0.0]`     |
   | Banana   | `[0.0, 0.0, 1.0]`     |



#### Import and creating SparkSession

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Python Spark create RDD example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()


In [0]:
df = spark.createDataFrame([
                            (0, "a"),
                            (1, "b"),
                            (2, "c"),
                            (3, "a"),
                            (4, "a"),
                            (5, "c")
                            ], ["id", "category"])
df.show()


+---+--------+
| id|category|
+---+--------+
|  0|       a|
|  1|       b|
|  2|       c|
|  3|       a|
|  4|       a|
|  5|       c|
+---+--------+



#### OneHotEncoder

##### Codificación de Categoría a Índice Numérico

In [0]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

# Codificación de la categoría a índice numérico
# Se utiliza StringIndexer para convertir la columna de categoría en índices numéricos
stringIndexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
model = stringIndexer.fit(df)  # Ajustar el modelo a los datos
indexed = model.transform(df)    # Transformar el DataFrame original

# Codificación One-Hot, se debe ajustar y luego transformar
encoder = OneHotEncoder(inputCol="categoryIndex", outputCol="categoryVec")
encoder_model = encoder.fit(indexed)  # Ajuste del modelo de codificación
encoded = encoder_model.transform(indexed)  # Transformación de los datos

# Mostrar el DataFrame codificado
encoded.show()


+---+--------+-------------+-------------+
| id|category|categoryIndex|  categoryVec|
+---+--------+-------------+-------------+
|  0|       a|          0.0|(2,[0],[1.0])|
|  1|       b|          2.0|    (2,[],[])|
|  2|       c|          1.0|(2,[1],[1.0])|
|  3|       a|          0.0|(2,[0],[1.0])|
|  4|       a|          0.0|(2,[0],[1.0])|
|  5|       c|          1.0|(2,[1],[1.0])|
+---+--------+-------------+-------------+



##### Pipeline para Codificación y Vectorización

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

# Definir las columnas categóricas a indexar
categoricalCols = ['category']

# Crear una lista de indexadores para las columnas categóricas
indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c)) for c in categoricalCols]

# Crear una lista de OneHotEncoders para cada columna indexada
encoders = [OneHotEncoder(inputCol=indexer.getOutputCol(),
            outputCol="{0}_encoded".format(indexer.getOutputCol()), 
            dropLast=False) for indexer in indexers]

# Usar VectorAssembler para combinar las columnas codificadas en un vector de características
assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders], outputCol="features")

# Crear un Pipeline con los indexadores, encoders y el ensamblador
pipeline = Pipeline(stages=indexers + encoders + [assembler])

# Ajustar el modelo del pipeline a los datos
model = pipeline.fit(df)

# Transformar los datos usando el modelo del pipeline
data = model.transform(df)


##### Mostrar resultados

In [0]:
data.show()

+---+--------+----------------+------------------------+-------------+
| id|category|category_indexed|category_indexed_encoded|     features|
+---+--------+----------------+------------------------+-------------+
|  0|       a|             0.0|           (3,[0],[1.0])|[1.0,0.0,0.0]|
|  1|       b|             2.0|           (3,[2],[1.0])|[0.0,0.0,1.0]|
|  2|       c|             1.0|           (3,[1],[1.0])|[0.0,1.0,0.0]|
|  3|       a|             0.0|           (3,[0],[1.0])|[1.0,0.0,0.0]|
|  4|       a|             0.0|           (3,[0],[1.0])|[1.0,0.0,0.0]|
|  5|       c|             1.0|           (3,[1],[1.0])|[0.0,1.0,0.0]|
+---+--------+----------------+------------------------+-------------+



### 10. Scale

El **escalado de características** es una técnica crucial en el preprocesamiento de datos de machine learning. Ayuda a normalizar las características para mejorar el rendimiento y la precisión de los modelos. En PySpark, hay varias técnicas de escalado que se pueden aplicar:


In [0]:
from pyspark.ml.feature import Normalizer, StandardScaler, MinMaxScaler,MaxAbsScaler
scaler_type = 'Normal'
if scaler_type=='Normal':
    scaler = Normalizer(inputCol="features", outputCol="scaledFeatures", p=1.0)
elif scaler_type=='Standard':
    scaler = StandardScaler(inputCol="features",
                            outputCol="scaledFeatures", withStd=True, withMean=False)
elif scaler_type=='MinMaxScaler':
    scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
elif scaler_type=='MaxAbsScaler':
    scaler = MaxAbsScaler(inputCol="features", outputCol="scaledFeatures")

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
df = spark.createDataFrame([(0, Vectors.dense([1.0, 0.5, -1.0]),),
                            (1, Vectors.dense([2.0, 1.0, 1.0]),),
                            (2, Vectors.dense([4.0, 10.0, 2.0]),)
                            ], ["id", "features"])
df.show()
pipeline = Pipeline(stages=[scaler])
model =pipeline.fit(df)
data = model.transform(df)
data.show()


+---+--------------+
| id|      features|
+---+--------------+
|  0|[1.0,0.5,-1.0]|
|  1| [2.0,1.0,1.0]|
|  2|[4.0,10.0,2.0]|
+---+--------------+

+---+--------------+------------------+
| id|      features|    scaledFeatures|
+---+--------------+------------------+
|  0|[1.0,0.5,-1.0]|    [0.4,0.2,-0.4]|
|  1| [2.0,1.0,1.0]|   [0.5,0.25,0.25]|
|  2|[4.0,10.0,2.0]|[0.25,0.625,0.125]|
+---+--------------+------------------+



#### Normalizer

**Normalizer** es una técnica de preprocesamiento utilizada para escalar cada vector de características de modo que tenga una **norma (longitud) de 1**. Esto se realiza dividiendo cada valor del vector por la longitud total del vector, manteniendo las proporciones relativas entre las características. Esta técnica es particularmente útil cuando se quiere mantener la **dirección** del vector de características, pero no su magnitud.

#### ¿Cómo Funciona?

**Normalizer** ajusta los datos dividiendo cada valor del vector por su norma. La norma es una medida de la magnitud del vector. Normalmente, se utiliza la **norma Euclidiana (L2)** o la **norma Manhattan (L1)**.

- **Fórmula General de Normalización**:
  
  `normalized_vector = vector / ||vector||`


  Donde `||vector||` representa la norma del vector

- **Norma \(L^2\) (Euclidiana)**:
  $$ 
  \|\text{vector}\|_2 = \sqrt{\sum_{i=1}^n x_i^2} 
  $$
  Esta norma se utiliza para garantizar que la longitud del vector sea 1.

- **Norma \(L^1\) (Manhattan)**:
  $$
  \|\text{vector}\|_1 = \sum_{i=1}^n |x_i|
  $$
  Esta norma se usa cuando queremos normalizar usando la suma de los valores absolutos.


In [0]:
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors

# Crear un DataFrame de ejemplo con vectores de características
dataFrame = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.5, -1.0])),
    (1, Vectors.dense([2.0, 1.0, 1.0])),
    (2, Vectors.dense([4.0, 10.0, 2.0]))
], ["id", "features"])

# Normalizar cada vector usando la norma L1 (Manhattan)
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)
l1NormData = normalizer.transform(dataFrame)
print("Normalizado usando la norma L1")
l1NormData.show()

# Normalizar cada vector usando la norma L∞ (Máximo)
lInfNormData = normalizer.transform(dataFrame, {normalizer.p: float("inf")})
print("Normalizado usando la norma L∞")
lInfNormData.show()


Normalizado usando la norma L1
+---+--------------+------------------+
| id|      features|      normFeatures|
+---+--------------+------------------+
|  0|[1.0,0.5,-1.0]|    [0.4,0.2,-0.4]|
|  1| [2.0,1.0,1.0]|   [0.5,0.25,0.25]|
|  2|[4.0,10.0,2.0]|[0.25,0.625,0.125]|
+---+--------------+------------------+

Normalizado usando la norma L∞
+---+--------------+--------------+
| id|      features|  normFeatures|
+---+--------------+--------------+
|  0|[1.0,0.5,-1.0]|[1.0,0.5,-1.0]|
|  1| [2.0,1.0,1.0]| [1.0,0.5,0.5]|
|  2|[4.0,10.0,2.0]| [0.4,1.0,0.2]|
+---+--------------+--------------+



#### StandardScaler

**StandardScaler** es una técnica de preprocesamiento utilizada para **escalar** y **centrar** las características de un dataset de modo que tengan una **media de 0** y una **desviación estándar de 1**. Esto se logra restando la media de cada característica y dividiendo por la desviación estándar. Es particularmente útil cuando se desea que todas las características tengan la misma importancia y escala.

#### ¿Cómo Funciona?

**StandardScaler** ajusta los datos haciendo que tengan una media de 0 y una desviación estándar de 1. Esta técnica ayuda a evitar que características con rangos de valores mayores dominen a otras durante el proceso de aprendizaje.

- **Fórmula de Estandarización**:

  `scaled_value = (x - μ) / σ`

  Donde:
  - `x` es el valor de la característica original.
  - `μ` es la **media** de la característica.
  - `σ` es la **desviación estándar** de la característica.

- **Proceso**:
  1. **Restar la Media**: Primero, se resta la media de cada característica, centrando los datos alrededor de 0.
  2. **Dividir por la Desviación Estándar**: Luego, se divide cada característica por su desviación estándar para garantizar que todas tengan una varianza uniforme.

#### Uso Común de StandardScaler

- **Modelos Basados en Gradiente**:
  - StandardScaler es útil para algoritmos como **regresión lineal** o **k-means**, que son sensibles a la escala de los datos. Ayuda a garantizar que cada característica tenga el mismo rango de influencia en el modelo.
  
- **PCA (Análisis de Componentes Principales)**:
  - En técnicas como **PCA**, StandardScaler asegura que todas las características contribuyan de manera equitativa al análisis de la varianza.


In [0]:
from pyspark.ml.feature import StandardScaler
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors

# Crear una sesión de Spark
spark = SparkSession.builder.appName("StandardScalerExample").getOrCreate()

data = [
    (0, Vectors.dense([1.0, 0.5, -1.0])),
    (1, Vectors.dense([2.0, 1.0, 1.0])),
    (2, Vectors.dense([4.0, 10.0, 2.0]))
]
df = spark.createDataFrame(data, ["id", "features"])

# Crear el escalador
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withMean=True, withStd=True)

# Ajustar el modelo del escalador a los datos
scaler_model = scaler.fit(df)

# Transformar los datos con el modelo ajustado
scaled_data = scaler_model.transform(df)

# Mostrar los resultados
scaled_data.select("features", "scaledFeatures").show(truncate=False)

+--------------+--------------------------------------------------------------+
|features      |scaledFeatures                                                |
+--------------+--------------------------------------------------------------+
|[1.0,0.5,-1.0]|[-0.8728715609439696,-0.6234796863885498,-1.0910894511799618] |
|[2.0,1.0,1.0] |[-0.21821789023599245,-0.5299577334302673,0.21821789023599242]|
|[4.0,10.0,2.0]|[1.0910894511799618,1.1534374198188169,0.8728715609439697]    |
+--------------+--------------------------------------------------------------+



#### MinMaxScaler

##### ¿Cómo Funciona?

**MinMaxScaler** transforma los datos al llevar cada característica dentro de un rango específico, por lo general entre 0 y 1.

- **Fórmula de Escalado con MinMaxScaler**:

  `scaled_value = (x - min) / (max - min)`

  Donde:
  - `x` es el valor original de la característica.
  - `min` es el valor mínimo de la característica en el dataset.
  - `max` es el valor máximo de la característica en el dataset.

##### Propósito

- **Uniformidad**: Garantiza que todas las características se encuentren dentro de un mismo rango de valores, lo cual puede ser muy beneficioso para algoritmos basados en distancia (como K-Means) y redes neuronales.
- **Preservación de las Relaciones**: Al contrario de la estandarización, **MinMaxScaler** no cambia la distribución relativa de los datos, solo ajusta los valores para que se encuentren dentro del rango especificado.


In [0]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors

# Crear un DataFrame de ejemplo con vectores de características
dataFrame = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.5, -1.0])),
    (1, Vectors.dense([2.0, 1.0, 1.0])),
    (2, Vectors.dense([4.0, 10.0, 2.0]))
], ["id", "features"])

# Crear MinMaxScaler y configurarlo para escalar los datos entre 0 y 1
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

# Ajustar el escalador a los datos y luego transformar los datos
scaler_model = scaler.fit(dataFrame)
scaled_data = scaler_model.transform(dataFrame)

# Mostrar los datos escalados
scaled_data.show(truncate=False)

+---+--------------+-----------------------------------------------------------+
|id |features      |scaledFeatures                                             |
+---+--------------+-----------------------------------------------------------+
|0  |[1.0,0.5,-1.0]|(3,[],[])                                                  |
|1  |[2.0,1.0,1.0] |[0.3333333333333333,0.05263157894736842,0.6666666666666666]|
|2  |[4.0,10.0,2.0]|[1.0,1.0,1.0]                                              |
+---+--------------+-----------------------------------------------------------+



### 11. PCA

**PCA (Principal Component Analysis)** es una técnica de reducción de dimensionalidad que se utiliza para **transformar** un conjunto de características posiblemente correlacionadas en un conjunto menor de **componentes principales**, que son combinaciones lineales de las características originales. Esto se logra de manera que se conserve la mayor cantidad de **varianza** posible de los datos originales.

#### ¿Qué es PCA?

El propósito principal de **PCA** es reducir la **dimensionalidad** del dataset para simplificar el análisis y la visualización, mientras se conserva la **información más relevante**. Los componentes principales se eligen de manera que sean **ortogonales** (sin correlación) entre sí y que cada componente represente la mayor cantidad de varianza posible.

#### Cómo Funciona PCA

- **Varianza**: PCA proyecta los datos en un nuevo espacio en el cual la varianza de los datos es máxima. 
- **Componentes Principales**:
  - Cada componente principal es una combinación lineal de las características originales.
  - El primer componente principal es la dirección que tiene la mayor varianza.
  - El segundo componente principal es la dirección ortogonal al primer componente y con la siguiente mayor varianza, y así sucesivamente.

#### Fórmula General

- Supongamos que tenemos un conjunto de datos \( X \) con \( n \) características.
- PCA transforma los datos mediante una combinación lineal:
  \[
  Z = XW
  \]
  Donde:
  - \( X \) es la matriz original de datos.
  - \( W \) es la matriz de pesos (autovectores).
  - \( Z \) es la nueva matriz de componentes principales.

#### Beneficios de PCA

1. **Reducción de Dimensionalidad**:
   - Ayuda a **simplificar** los datos, lo cual es particularmente útil cuando tenemos un gran número de características, y muchas de ellas están correlacionadas.
  
2. **Evita la Colinealidad**:
   - Debido a que los componentes principales son **ortogonales** entre sí, **PCA** elimina la **colinealidad** entre las características.

3. **Aumenta la Eficiencia del Modelo**:
   - Al reducir el número de características, se reduce el **costo computacional**, lo cual mejora la eficiencia de los modelos de aprendizaje automático.


In [0]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("PCAExample").getOrCreate()

# Crear un DataFrame con vectores de características
dataFrame = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.5, -1.0])),
    (1, Vectors.dense([2.0, 1.0, 1.0])),
    (2, Vectors.dense([4.0, 10.0, 2.0]))
], ["id", "features"])

# Crear el PCA, reduciendo a 2 componentes principales
pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")

# Ajustar el modelo de PCA y transformar los datos
pca_model = pca.fit(dataFrame)
pca_result = pca_model.transform(dataFrame)

# Mostrar los resultados
pca_result.select("id", "pcaFeatures").show(truncate=False)


+---+-----------------------------------------+
|id |pcaFeatures                              |
+---+-----------------------------------------+
|0  |[-0.5112783832867785,-0.685424930259303] |
|1  |[-1.6835922521434263,1.2832497998285195] |
|2  |[-10.884223309499816,0.12204863447901526]|
+---+-----------------------------------------+



### 14. DCT

**DCT (Discrete Cosine Transform)** o Transformada Discreta del Coseno es una técnica utilizada para **transformar datos en el dominio del tiempo** o **espacio** a un dominio de **frecuencias**. La DCT es ampliamente utilizada en **procesamiento de señales**, **compresión de imágenes**, y **machine learning** para capturar patrones que no son fácilmente visibles en los datos originales.

#### ¿Qué es DCT?

La **Transformada Discreta del Coseno** transforma un vector de características en un nuevo conjunto de valores que representan la amplitud de las frecuencias en los datos. La **DCT** convierte un **vector** de valores en una combinación de funciones coseno que representan las diferentes **frecuencias** presentes en el vector. Se suele utilizar cuando es necesario analizar los datos en términos de sus componentes de frecuencia, para simplificar el análisis o la compresión.

#### Cómo Funciona DCT

- **Transformación de Frecuencia**: La DCT toma un conjunto de datos en el dominio del tiempo o espacio y calcula una serie de **coeficientes** que representan las frecuencias que componen los datos.
- **Componentes de Baja y Alta Frecuencia**:
  - La DCT transforma los datos en componentes de **baja frecuencia** y **alta frecuencia**, lo cual es útil para analizar tendencias o patrones subyacentes.

#### Fórmula de la Transformación DCT

La fórmula de la **DCT tipo II** (que es la más común) para un vector de tamaño \( N \) es:

$$
X_k = \sum_{n=0}^{N-1} x_n \cos\left(\frac{\pi k (2n + 1)}{2N}\right)
$$

Donde:
- \( x_n \) son los valores originales del vector.
- \( X_k \) es el coeficiente del coseno para la frecuencia \( k \).

#### Beneficios de la DCT

1. **Compresión de Información**:
   - **DCT** es ampliamente utilizada en **compresión de imágenes** y **audio** (como JPEG y MP3) ya que permite retener la información más relevante en **pocos coeficientes**.

2. **Reducción de Ruido**:
   - Los **componentes de alta frecuencia** suelen representar ruido, por lo que al transformar los datos y quedarnos solo con los primeros coeficientes, se puede **reducir el ruido**.

3. **Mejor Representación**:
   - Al transformar los datos en el dominio de frecuencia, se pueden identificar **patrones** que no son evidentes en los datos originales.




In [0]:
from pyspark.ml.feature import DCT
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("DCTExample").getOrCreate()

# Crear un DataFrame con vectores de características
dataFrame = spark.createDataFrame([
    (0, Vectors.dense([1.0, 2.0, 3.0, 4.0])),
    (1, Vectors.dense([4.0, 3.0, 2.0, 1.0])),
    (2, Vectors.dense([0.0, 1.0, -1.0, -2.0]))
], ["id", "features"])

# Crear la transformación DCT
dct = DCT(inverse=False, inputCol="features", outputCol="dctFeatures")

# Transformar los datos con DCT
dct_data = dct.transform(dataFrame)

# Mostrar los resultados
dct_data.select("id", "dctFeatures").show(truncate=False)

+---+-----------------------------------------------------------------+
|id |dctFeatures                                                      |
+---+-----------------------------------------------------------------+
|0  |[5.0,-2.2304424973876635,0.0,-0.15851266778110737]               |
|1  |[5.0,2.2304424973876635,0.0,0.15851266778110737]                 |
|2  |[-1.0,1.8477590650225737,-1.0000000000000002,-0.7653668647301795]|
+---+-----------------------------------------------------------------+



## Feature Select

**Feature Selection** o **Selección de Características** es una técnica clave en el preprocesamiento de datos para **reducir el número de características** que se utilizarán en un modelo de machine learning. El objetivo es seleccionar solo las características más relevantes que maximicen el rendimiento del modelo, reduciendo el **ruido** y evitando el **sobreajuste (overfitting)**.


### 1. LASSO

**LASSO** es un tipo de regresión lineal que añade un término de penalización basado en la **suma de los valores absolutos de los coeficientes**. La penalización L1 es una forma de regularización que tiende a reducir el valor de los coeficientes. Al aumentar la fuerza de la penalización, algunos coeficientes se reducen a exactamente **cero**, y esto se utiliza para identificar y eliminar características irrelevantes.

- **Penalización L1**:
  - El término de penalización es proporcional a la **suma de los valores absolutos** de los coeficientes, es decir:
  $$
  \text{Lasso Loss} = \text{RSS} + \lambda \sum_{j=1}^{p} |\beta_j|
  $$
  Donde:
  - `RSS` es el **Residual Sum of Squares** (suma de los cuadrados de los residuos).
  - `λ` es el **parámetro de regularización** que controla la fuerza de la penalización.
  - `β_j` representa los coeficientes del modelo.

#### ¿Por Qué Usar LASSO para Selección de Características?

1. **Eliminar Características Irrelevantes**:
   - LASSO ajusta algunos coeficientes a cero, eliminando efectivamente las características que no contribuyen significativamente al modelo.

2. **Reducir el Sobreajuste**:
   - Al eliminar características innecesarias y reducir los coeficientes, LASSO reduce la complejidad del modelo, lo cual ayuda a prevenir el **sobreajuste**.

3. **Mejora de la Interpretabilidad**:
   - Con menos características en el modelo, es más fácil **interpretar** y **entender** el impacto de cada característica en la predicción.


In [0]:
# Importa las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import LinearRegression

# Inicializa la sesión de Spark
spark = SparkSession.builder.appName("LASSO_Iris_Regression").getOrCreate()

# Creación de un DataFrame simulado con el dataset Iris
data = [
    (5.1, 3.5, 1.4, 0.2, "setosa"),
    (4.9, 3.0, 1.4, 0.2, "setosa"),
    (6.2, 3.4, 5.4, 2.3, "virginica"),
    (5.9, 3.0, 5.1, 1.8, "virginica"),
    (5.4, 3.4, 1.7, 0.2, "setosa"),
    (6.5, 2.8, 4.6, 1.5, "versicolor"),
    (5.7, 2.8, 4.5, 1.3, "versicolor"),
    # (Agrega más filas del dataset Iris completo si es necesario)
]
columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]

# Crea el DataFrame de Spark con los datos de Iris
df = spark.createDataFrame(data, columns)

# Muestra una vista previa del DataFrame original
print("DataFrame Original:")
df.show(5)

# Convertir la columna de clase "species" en un índice numérico para usar en el modelo de regresión
indexer = StringIndexer(inputCol="species", outputCol="label")
indexed_df = indexer.fit(df).transform(df)

# Combina las características en un solo vector llamado "features"
assembler = VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol="features")
assembled_df = assembler.transform(indexed_df)

# Define el modelo de regresión lineal con regularización LASSO (L1 > 0)
lasso = LinearRegression(featuresCol="features", labelCol="label", regParam=0.1, elasticNetParam=1.0)

# Entrena el modelo
lasso_model = lasso.fit(assembled_df)

# Imprime los coeficientes para ver cuáles se han reducido a cero
print("Coeficientes:", lasso_model.coefficients)
print("Intercepto:", lasso_model.intercept)

# Opcional: Muestra las predicciones para los datos de entrada
predictions = lasso_model.transform(assembled_df)
print("Predicciones:")
predictions.select("features", "label", "prediction").show()


DataFrame Original:
+------------+-----------+------------+-----------+---------+
|sepal_length|sepal_width|petal_length|petal_width|  species|
+------------+-----------+------------+-----------+---------+
|         5.1|        3.5|         1.4|        0.2|   setosa|
|         4.9|        3.0|         1.4|        0.2|   setosa|
|         6.2|        3.4|         5.4|        2.3|virginica|
|         5.9|        3.0|         5.1|        1.8|virginica|
|         5.4|        3.4|         1.7|        0.2|   setosa|
+------------+-----------+------------+-----------+---------+
only showing top 5 rows

Coeficientes: [0.0,0.0,0.0,0.8797878708974681]
Intercepto: -0.0854870045330018
Predicciones:
+-----------------+-----+-------------------+
|         features|label|         prediction|
+-----------------+-----+-------------------+
|[5.1,3.5,1.4,0.2]|  0.0|0.09047056964649185|
|[4.9,3.0,1.4,0.2]|  0.0|0.09047056964649185|
|[6.2,3.4,5.4,2.3]|  2.0| 1.9380250985311747|
|[5.9,3.0,5.1,1.8]|  2.0| 1.

### 2. RandomForestClassifier


**Random Forest** es un conjunto de **árboles de decisión** entrenados de manera independiente. La predicción final se realiza mediante la combinación (votación) de las predicciones individuales de cada árbol. Esta técnica se destaca por:

- Ser **robusta frente al sobreajuste** debido al promedio de las predicciones de muchos árboles.
- Ser **flexible**, capaz de manejar características categóricas, numéricas, y grandes volúmenes de datos.

#### Selección de Características con Random Forest

**Random Forest** también sirve para la **selección de características**, ya que puede medir la **importancia** de cada característica basándose en cómo contribuye a las decisiones de los árboles durante la clasificación. Las características más importantes serán aquellas que más veces se utilicen para dividir nodos y que mejor separen las clases.

- **Importancia de Característica**: Cada característica se mide según su contribución para reducir el **índice de impureza** en los nodos de los árboles.
- **Índice de Gini**: En los árboles de decisión, el **índice de Gini** se utiliza para medir la impureza de una partición, es decir, cuán mezcladas están las clases en un nodo.

La **importancia de una característica** en Random Forest se calcula basándose en la cantidad que contribuye cada característica para reducir el índice de Gini en las divisiones de los árboles. A mayor reducción de impureza, mayor es la importancia de la característica.

#### Fórmula del Índice de Gini

El **índice de Gini** para una partición se calcula como:

$$
Gini = 1 - \sum_{i=1}^{C} p_i^2
$$

Donde:

- `C` es el número de clases.
- `p_i` es la proporción de observaciones de la clase `i` en el nodo.

Un valor de **Gini** cercano a 0 indica baja impureza (es decir, un nodo con observaciones de la misma clase), mientras que un valor cercano a 1 indica alta impureza.


In [0]:
# Importa las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Inicializa la sesión de Spark
spark = SparkSession.builder.appName("RandomForest_Iris_Classification").getOrCreate()

# Creación de un DataFrame simulado con el dataset Iris
data = [
    (5.1, 3.5, 1.4, 0.2, "setosa"),
    (4.9, 3.0, 1.4, 0.2, "setosa"),
    (6.2, 3.4, 5.4, 2.3, "virginica"),
    (5.9, 3.0, 5.1, 1.8, "virginica"),
    (5.4, 3.4, 1.7, 0.2, "setosa"),
    (6.5, 2.8, 4.6, 1.5, "versicolor"),
    (5.7, 2.8, 4.5, 1.3, "versicolor"),
    # (Agrega más filas del dataset Iris completo si es necesario)
]
columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]

# Crea el DataFrame de Spark con los datos de Iris
df = spark.createDataFrame(data, columns)

# Muestra una vista previa del DataFrame original
print("DataFrame Original:")
df.show(5)

# Convertir la columna de clase "species" en un índice numérico
indexer = StringIndexer(inputCol="species", outputCol="label")
indexed_df = indexer.fit(df).transform(df)

# Combina las características en un solo vector llamado "features"
assembler = VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol="features")
assembled_df = assembler.transform(indexed_df)

# Define el modelo de clasificación de Random Forest
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=100, maxDepth=5)

# Entrena el modelo de Random Forest
rf_model = rf.fit(assembled_df)

# Realiza predicciones en el mismo conjunto de datos
predictions = rf_model.transform(assembled_df)

# Mostrar las predicciones
print("Predicciones:")
predictions.select("features", "label", "prediction").show(5)

# Evaluar el modelo usando la precisión
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Precisión del modelo: {accuracy}")

# Importancia de las características
print("Importancia de las características:", rf_model.featureImportances)


DataFrame Original:
+------------+-----------+------------+-----------+---------+
|sepal_length|sepal_width|petal_length|petal_width|  species|
+------------+-----------+------------+-----------+---------+
|         5.1|        3.5|         1.4|        0.2|   setosa|
|         4.9|        3.0|         1.4|        0.2|   setosa|
|         6.2|        3.4|         5.4|        2.3|virginica|
|         5.9|        3.0|         5.1|        1.8|virginica|
|         5.4|        3.4|         1.7|        0.2|   setosa|
+------------+-----------+------------+-----------+---------+
only showing top 5 rows

Predicciones:
+-----------------+-----+----------+
|         features|label|prediction|
+-----------------+-----+----------+
|[5.1,3.5,1.4,0.2]|  0.0|       0.0|
|[4.9,3.0,1.4,0.2]|  0.0|       0.0|
|[6.2,3.4,5.4,2.3]|  2.0|       2.0|
|[5.9,3.0,5.1,1.8]|  2.0|       2.0|
|[5.4,3.4,1.7,0.2]|  0.0|       0.0|
+-----------------+-----+----------+
only showing top 5 rows

Precisión del modelo: 1.0

### 3. Chi-Square


La **prueba Chi-Cuadrado** se utiliza para determinar si existe una relación significativa entre cada característica y la **etiqueta de salida**. Se calcula un valor `χ^2` que indica cuán dependientes son las características respecto a la etiqueta. Las características con valores `χ^2` más altos son las más importantes y, por lo tanto, se seleccionan para formar el conjunto final.

- **Valor Chi-Cuadrado (`χ^2`)**:
  - Este valor mide cuán esperados o inesperados son los datos en comparación con una hipótesis nula.
  - Un valor alto indica que existe una relación fuerte entre la característica y la variable de salida.

##### Fórmula del Test Chi-Cuadrado

La fórmula para calcular el valor `χ^2` es:

$$
\chi^2 = \sum \frac{(O_i - E_i)^2}{E_i}
$$

Donde:

- `O_i` es el valor **observado**.
- `E_i` es el valor **esperado** bajo la hipótesis nula (que asume que no hay relación entre las variables).

El valor calculado de `χ^2` se compara con un valor crítico basado en el nivel de significancia elegido y los **grados de libertad** del problema, para decidir si una característica tiene una relación significativa con la etiqueta.

In [0]:

from pyspark.ml.feature import ChiSqSelector, StringIndexer, VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("ChiSquareFeatureSelectionExample").getOrCreate()

# Crear un DataFrame con características de empleados
data = [
    (1, "Licenciatura", 30, "Ventas", 1.0),
    (0, "Maestría", 25, "IT", 0.0),
    (1, "Secundaria", 40, "Ventas", 1.0),
    (0, "Licenciatura", 20, "IT", 0.0),
    (1, "Maestría", 50, "Finanzas", 1.0),
    (0, "Secundaria", 15, "Finanzas", 0.0)
]
columns = ["id", "educacion", "horas_capacitacion", "departamento", "promocion"]

# Crear un DataFrame con los datos de ejemplo
df = spark.createDataFrame(data, columns)

# Convertir columnas categóricas a índices numéricos usando StringIndexer
indexer_educacion = StringIndexer(inputCol="educacion", outputCol="educacion_index")
indexer_departamento = StringIndexer(inputCol="departamento", outputCol="departamento_index")
df_indexed = indexer_educacion.fit(df).transform(df)
df_indexed = indexer_departamento.fit(df_indexed).transform(df_indexed)

# Combinar las características en un vector con VectorAssembler
assembler = VectorAssembler(inputCols=["educacion_index", "horas_capacitacion", "departamento_index"], outputCol="features")
assembled_df = assembler.transform(df_indexed)

# Aplicar ChiSqSelector para seleccionar las características más importantes
selector = ChiSqSelector(numTopFeatures=2, featuresCol="features", outputCol="selectedFeatures", labelCol="promocion")

# Ajustar y transformar los datos
result = selector.fit(assembled_df).transform(assembled_df)

# Mostrar los resultados de las características seleccionadas
result.select("features", "selectedFeatures").show(truncate=False)


+--------------+----------------+
|features      |selectedFeatures|
+--------------+----------------+
|[0.0,30.0,2.0]|[30.0,2.0]      |
|[1.0,25.0,1.0]|[25.0,1.0]      |
|[2.0,40.0,2.0]|[40.0,2.0]      |
|[0.0,20.0,1.0]|[20.0,1.0]      |
|[1.0,50.0,0.0]|[50.0,0.0]      |
|[2.0,15.0,0.0]|[15.0,0.0]      |
+--------------+----------------+

